# MNIST classifier using CNN

## Authors:

Nicola Rocco de Luna Pedulla - nrlp


Renan de Freitas Lins - rfl3

Ricardo Rossiter Barioni - rrb

Victor Miranda de Melo - vmm

## Firstly, import the required dependencies

GPU is not being used

In [1]:
from DeepLearningPython35.network3 import load_data_shared, Network, ConvPoolLayer, FullyConnectedLayer, SoftmaxLayer, ReLU

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
c:\python 3.5\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Running with a CPU.  If this is not desired, then the modify network3.py to set
the GPU flag to True.


In [2]:
training_data, validation_data, test_data = load_data_shared()
mini_batch_size = 10

In [ ]:
net = Network([
    ConvPoolLayer(image_shape=(mini_batch_size, 1, 28, 28),
                  filter_shape=(20, 1, 5, 5),
                  poolsize=(2, 2),
                  activation_fn=ReLU),
    ConvPoolLayer(image_shape=(mini_batch_size, 20, 12, 12),
                  filter_shape=(40, 20, 5, 5),
                  poolsize=(2, 2),
                  activation_fn=ReLU),
    FullyConnectedLayer(n_in=40*4*4, n_out=100, activation_fn=ReLU),
    SoftmaxLayer(n_in=100, n_out=10)], mini_batch_size)
net.SGD(training_data, 1, mini_batch_size, 0.03, validation_data, test_data, lmbda=0.1)

c:\python 3.5\lib\site-packages\theano\tensor\nnet\conv.py:98: UserWarning: theano.tensor.nnet.conv.conv2d is deprecated. Use theano.tensor.nnet.conv2d instead.
  warnings.warn("theano.tensor.nnet.conv.conv2d is deprecated."


Training mini-batch number 0
Training mini-batch number 1000
Training mini-batch number 2000
Training mini-batch number 3000
Training mini-batch number 4000
Epoch 0: validation accuracy 97.47%
This is the best validation accuracy to date.


In [ ]:
net